In [12]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor

dfTrain = pd.read_csv('../data/TRAIN_TEST_corrected2/train_corrected2.csv')
dfTest = pd.read_csv('../data/TRAIN_TEST_corrected2/test_corrected2.csv')

In [13]:
dfTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004777 entries, 0 to 1004776
Data columns (total 14 columns):
id                           1004777 non-null object
year_created                 1004777 non-null float64
month_created                1004777 non-null float64
day_created                  1004777 non-null float64
distance_to_obelisco_mtrs    1004777 non-null float64
lat                          1004777 non-null float64
lon                          1004777 non-null float64
surface_total_in_m2          1004777 non-null float64
rooms                        1004777 non-null float64
price_usd                    1004777 non-null float64
PH                           1004777 non-null float64
apartment                    1004777 non-null float64
house                        1004777 non-null float64
store                        1004777 non-null float64
dtypes: float64(13), object(1)
memory usage: 107.3+ MB


In [14]:
dfTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 13 columns):
id                           14166 non-null int64
year_created                 14166 non-null float64
month_created                14166 non-null float64
day_created                  14166 non-null float64
distance_to_obelisco_mtrs    14166 non-null float64
lat                          14166 non-null float64
lon                          14166 non-null float64
surface_total_in_m2          14166 non-null float64
rooms                        14166 non-null float64
PH                           14166 non-null float64
apartment                    14166 non-null float64
house                        14166 non-null float64
store                        14166 non-null float64
dtypes: float64(12), int64(1)
memory usage: 1.4 MB


In [15]:
target = dfTrain.price_usd
testIds = dfTest.id

norm = preprocessing.Normalizer()
train = norm.fit_transform(dfTrain.drop(['id', 'price_usd'], axis=True))

testVals = norm.transform(dfTest.drop(['id'], axis=True))

In [16]:
train = pd.DataFrame(train, columns=['year_created', 'month_created', 'day_created', 
                                     'distance_to_obelisco_mtrs', 'lat', 'lon', 
                                     'surface_total_in_m2', 'rooms', 'PH', 
                                     'apartment', 'house', 'store'])
train.head(3)

,year_created,month_created,day_created,distance_to_obelisco_mtrs,lat,lon,surface_total_in_m2,rooms,PH,apartment,house,store
0,0.118544,0.000471,0.001649,0.992933,-0.002038,-0.003449,0.003651,0.000177,0.0,0.000059,0.000000,0.0
1,0.351536,0.001397,0.004890,0.936054,-0.006042,-0.010205,0.007684,0.000349,0.0,0.000175,0.000000,0.0
2,0.038434,0.000153,0.000535,0.999256,-0.000661,-0.001126,0.002864,0.000038,0.0,0.000000,0.000019,0.0


In [17]:
testVals = pd.DataFrame(testVals, columns=['year_created', 'month_created', 'day_created', 
                                           'distance_to_obelisco_mtrs', 'lat', 'lon', 
                                           'surface_total_in_m2', 'rooms', 'PH', 
                                           'apartment', 'house', 'store'])

In [18]:
testVals.head(3)

,year_created,month_created,day_created,distance_to_obelisco_mtrs,lat,lon,surface_total_in_m2,rooms,PH,apartment,house,store
0,0.737566,2.925397e-03,0.008776,0.674755,-0.012656,-0.021342,0.000000,1.097024e-03,0.0,3.656747e-04,0.0,0.0
1,0.000184,7.317018e-07,0.000002,1.000000,0.000003,0.000001,0.000000,1.829255e-07,0.0,9.146273e-08,0.0,0.0
2,0.523091,2.074731e-03,0.000259,0.851981,-0.008970,-0.015149,0.013745,7.780240e-04,0.0,2.593413e-04,0.0,0.0


In [19]:
knn = KNeighborsRegressor(n_neighbors=30, weights='distance', 
                          algorithm='kd_tree', leaf_size=30, 
                          metric='euclidean', n_jobs=-1)

knn.fit(train, target)

KNeighborsRegressor(algorithm='kd_tree', leaf_size=30, metric='euclidean',
          metric_params=None, n_jobs=-1, n_neighbors=30, p=2,
          weights='distance')

In [20]:
predictions = knn.predict(testVals)

In [21]:
result = pd.DataFrame({'id':testIds, 'price_usd':predictions})
result.head()

,id,price_usd
0,3632,78441.396952
1,3633,101953.365716
2,2263404,147741.261453
3,2263405,81392.830896
4,2263406,110329.124376


In [22]:
result.to_csv('../data/predictions/4th_acercamiento.csv', index=False)